In [16]:
import dask.dataframe as dd
import pandas as pd

def sloc_percent(row):
    row['sloc_percent'] = 100*(row.git_diff_src_churn.astype('float')/row.gh_sloc.astype('float'))
    row['gh_test_lines_per_kloc'] = row.gh_test_lines_per_kloc/1000
    return row

#df = dd.read_csv('data/travistorrent_11_1_2017.csv',blocksize=25000000,usecols=['tr_build_id','gh_project_name','git_num_all_built_commits','gh_is_pr','git_trigger_commit','gh_sloc','git_diff_src_churn','gh_diff_files_added','gh_diff_files_deleted','gh_diff_files_modified'])
df = dd.read_csv('data/travistorrent_11_1_2017.csv',blocksize=25000000,usecols=['tr_build_id','gh_project_name','git_num_all_built_commits','git_trigger_commit','gh_sloc','git_diff_src_churn', 'gh_test_lines_per_kloc'])
pdf = df.drop_duplicates().map_partitions(sloc_percent).compute()

In [23]:
pdf_one_line_change = pdf[pdf.git_diff_src_churn == 1]
#pdf_one_line_change.sort_values('gh_sloc',inplace=True,ascending=False)
pdf_one_line_change['git_diff_src_churn'].describe()

count    16159.0
mean         1.0
std          0.0
min          1.0
25%          1.0
50%          1.0
75%          1.0
max          1.0
Name: git_diff_src_churn, dtype: float64

In [27]:
print(16159/float(557211)*100) # one commit builds 
16159/float(680209)*100 # all builds

2.8999786436376884


2.375593383798215

Ergebnis: 2.9% aller Ein-Commit Builds sind Ein-Zeilen Changes. 2.4% aller Builds sind ein-Zeilen Changes.

TODO: Berechnung wie oben wo "alle" == Builds mit mindestens einem source change. Frage: Wieviele Source-Code-Change Builds haben nur eine geänderte Zeile.
TODO: Wieviele Builds haben har keinen Source change?
TODO: Gegenmenge 'zerlegen' (Test/Doc/...)
